# Check for test failures - Defects4j 2.0
This script is the second main script to be executed and it compares the gzoltar coverage results (that were be obtained via the auxiliary scripts) and the defects4j bug-triggering tests
Obs: In order to be able to execute this script, you must unzip the zip files that can be found in the locations data/gzoltar_files/ and data/gzoltar_files/Closure/

### Vars declaration

In [17]:
import os
from secrets import base_path


paths_dict=  {
        "bugs_with_stack_traces_details_file_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
        "gzoltar_files_path":  os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "gzoltar_files"),
        "output_file_path":  os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "rq1_results.json"),
        "zenodo_gzoltar_files_path": os.path.join(base_path, "program_spectra", "vBuggy"),
        "defects4j_path": os.path.join(base_path, "defects4j")
}

### Getting the failing tests information for each bug with stack traces

In [20]:
import utils
import importlib
importlib.reload(utils)

bugs_data = utils.json_file_to_dict(paths_dict["bugs_with_stack_traces_details_file_path"])
bugs_with_failing_tests_in_commons_with_defects4j = {}
bugs_without_failing_tests_in_commons_with_defects4j = {}
bugs_with_coverage_problem = {}
bugs_with_no_coverage = []
for project in bugs_data.keys():
        print("----- " + project + " -----")
        project_gzoltar_folder = os.path.join(paths_dict["gzoltar_files_path"], project)
        for bug_id in bugs_data[project].keys():
                bug_report_id = project+ "_" + bug_id
                bug_gzoltar_folder = os.path.join(project_gzoltar_folder, bug_id)
                if not os.path.exists(bug_gzoltar_folder):
                        bugs_with_no_coverage.append(bug_report_id)
                        continue
                tests = utils.read_tests_file(bug_gzoltar_folder)
                number_not_passing_tests = len(tests["failing_tests"]) + len(tests["compilation_problems"])
                percentage_not_passing_tests = number_not_passing_tests / (number_not_passing_tests + len(tests["passing_tests"]))
                if len(tests["passing_tests"]) == 0 or percentage_not_passing_tests>0.1:
                        bugs_with_coverage_problem[bug_report_id] = {
                                "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                                "number_of_failing_tests": len(tests["failing_tests"]),
                                "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                                "failing_tests": tests["failing_tests"]
                        }
                        continue
                print(bug_report_id + ": "+ str(len(tests["failing_tests"])))
                found = False
                bugs_in_common_defects4j = []
                for failing_test in tests["failing_tests"]:
                        for defects4j_test in bugs_data[project][bug_id]["defects4j_trigger_tests"]:
                                defects4j_test = defects4j_test.replace("::", "#")
                                if failing_test == defects4j_test:
                                        found = True
                                        bugs_in_common_defects4j.append(failing_test)
                if found:
                        bugs_with_failing_tests_in_commons_with_defects4j[bug_report_id] = {
                                "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                                "number_of_failing_tests": len(tests["failing_tests"]),
                                "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                                "failing_tests": tests["failing_tests"],
                                "failing_tests_in_common_with_defects4j": bugs_in_common_defects4j
                        }
                else:
                        bugs_without_failing_tests_in_commons_with_defects4j[bug_report_id] = {
                                "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                                "number_of_failing_tests": len(tests["failing_tests"]),
                                "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                                "failing_tests": tests["failing_tests"]
                        }

output_dict = {
        "bugs_with_stack_traces": {
                "bugs_with_no_coverage": bugs_with_no_coverage,
                "bugs_with_coverage_problem": bugs_with_coverage_problem,
                "bugs_with_failing_tests_in_commons_with_defects4j": bugs_with_failing_tests_in_commons_with_defects4j,
                "bugs_without_failing_tests_in_commons_with_defects4j": bugs_without_failing_tests_in_commons_with_defects4j
        }
}
print("___________")
print("bugs_with_no_coverage: " + str(len(bugs_with_no_coverage)))
print("bugs_with_coverage_problem: " + str(len(bugs_with_coverage_problem)))
percentage_with_fault_triggering_tests = len(bugs_with_failing_tests_in_commons_with_defects4j) / (len(bugs_without_failing_tests_in_commons_with_defects4j) + len(bugs_with_failing_tests_in_commons_with_defects4j)) * 100
percentage_with_fault_triggering_tests= "{:.2f}".format(percentage_with_fault_triggering_tests)
print(str(len(bugs_with_failing_tests_in_commons_with_defects4j)) + " out of " + str(len(bugs_without_failing_tests_in_commons_with_defects4j) + len(bugs_with_failing_tests_in_commons_with_defects4j)) + " bugs have tests in common with defects4j (" + str(percentage_with_fault_triggering_tests) + "%)")

utils.dict_to_json_file(paths_dict["output_file_path"], output_dict)

----- Cli -----
Cli_14: 1
Cli_5: 3
Cli_7: 2
----- Closure -----
Closure_106: 1
Closure_125: 4
Closure_143: 1
Closure_152: 2
Closure_2: 3
Closure_34: 2
Closure_37: 2
Closure_55: 2
----- Codec -----
Codec_8: 0
----- Collections -----
Collections_28: 0
----- Compress -----
Compress_1: 0
Compress_12: 0
Compress_14: 0
Compress_17: 0
Compress_18: 0
Compress_23: 1
Compress_24: 3
Compress_27: 2
Compress_31: 2
Compress_32: 1
Compress_34: 2
----- Csv -----
Csv_12: 0
Csv_4: 0
----- Gson -----
Gson_12: 0
Gson_5: 0
Gson_8: 0
----- JacksonCore -----
JacksonCore_10: 1
JacksonCore_11: 0
JacksonCore_4: 1
JacksonCore_7: 0
----- JacksonDatabind -----
JacksonDatabind_14: 26
JacksonDatabind_15: 25
JacksonDatabind_25: 32
JacksonDatabind_28: 35
JacksonDatabind_3: 27
JacksonDatabind_32: 33
JacksonDatabind_37: 32
JacksonDatabind_59: 0
JacksonDatabind_73: 35
JacksonDatabind_9: 26
----- Jsoup -----
Jsoup_27: 0
Jsoup_34: 0
Jsoup_54: 0
Jsoup_78: 0
Jsoup_80: 0
Jsoup_82: 0
Jsoup_84: 1
Jsoup_90: 0
----- JxPath -----


### Getting the failing tests information for each bug without stack traces

In [21]:
import glob

coverages_path = os.path.join(paths_dict["zenodo_gzoltar_files_path"], "*", "*")
bugs_with_failing_tests_in_commons_with_defects4j = {}
bugs_without_failing_tests_in_commons_with_defects4j = {}
bugs_with_coverage_problem = {}
bugs_with_no_coverage = []

for bug_coverage_folder in glob.glob(coverages_path):
        project = bug_coverage_folder.split("/")[-2]
        bug_id = bug_coverage_folder.split("/")[-1].split("_")[1]
        bug_report_id = project + "_" + bug_id
        if utils.bug_belongs_to_file(paths_dict["bugs_with_stack_traces_details_file_path"], project, bug_id):
                continue #It is a bug with stak traces

        #Getting defects4j trigger tests
        defects4j_trigger_tests = []
        path = os.path.join(paths_dict["defects4j_path"], "framework", "projects", project, "trigger_tests")
        lines = utils.read_file_lines(bug_id, path)
        for line in lines:
                if line.startswith("--- "):
                        failing_test_name = line.replace("--- ", "").replace("\n", "")
                        defects4j_trigger_tests.append(failing_test_name)

        bug_gzoltar_folder = os.path.join(bug_coverage_folder, "txt")
        tests = utils.read_tests_file(bug_gzoltar_folder)
        number_not_passing_tests = len(tests["failing_tests"]) + len(tests["compilation_problems"])
        no_tests = False
        try:
                percentage_not_passing_tests = number_not_passing_tests / (number_not_passing_tests + len(tests["passing_tests"]))
        except ZeroDivisionError:
                no_tests = True
        if no_tests or len(tests["passing_tests"]) == 0 or percentage_not_passing_tests > 0.1:
                bugs_with_coverage_problem[bug_report_id] = {
                        "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                        "number_of_failing_tests": len(tests["failing_tests"]),
                        "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                        "failing_tests": tests["failing_tests"]
                }
                continue
        print(bug_report_id + ": "+ str(len(tests["failing_tests"])))
        found = False
        bugs_in_common_defects4j = []
        for failing_test in tests["failing_tests"]:
                for defects4j_test in defects4j_trigger_tests:
                        defects4j_test = defects4j_test.replace("::", "#")
                        if failing_test == defects4j_test:
                                found = True
                                bugs_in_common_defects4j.append(failing_test)
        if found:
                bugs_with_failing_tests_in_commons_with_defects4j[bug_report_id] = {
                        "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                        "number_of_failing_tests": len(tests["failing_tests"]),
                        "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                        "failing_tests": tests["failing_tests"],
                        "failing_tests_in_common_with_defects4j": bugs_in_common_defects4j
                }
        else:
                bugs_without_failing_tests_in_commons_with_defects4j[bug_report_id] = {
                        "total_number_of_tests": number_not_passing_tests + len(tests["passing_tests"]),
                        "number_of_failing_tests": len(tests["failing_tests"]),
                        "number_of_tests_with_compilation_problems": len(tests["compilation_problems"]),
                        "failing_tests": tests["failing_tests"]
                }

output_dict = utils.json_file_to_dict(paths_dict["output_file_path"])
output_dict.update({
        "bugs_without_stack_traces": {
                "bugs_with_no_coverage": bugs_with_no_coverage,
                "bugs_with_coverage_problem": bugs_with_coverage_problem,
                "bugs_with_failing_tests_in_commons_with_defects4j": bugs_with_failing_tests_in_commons_with_defects4j,
                "bugs_without_failing_tests_in_commons_with_defects4j": bugs_without_failing_tests_in_commons_with_defects4j
        }
})
print("___________")
print("bugs_with_no_coverage: " + str(len(bugs_with_no_coverage)))
print("bugs_with_coverage_problem: " + str(len(bugs_with_coverage_problem)))
percentage_with_fault_triggering_tests = len(bugs_with_failing_tests_in_commons_with_defects4j) / (len(bugs_without_failing_tests_in_commons_with_defects4j) + len(bugs_with_failing_tests_in_commons_with_defects4j)) * 100
percentage_with_fault_triggering_tests= "{:.2f}".format(percentage_with_fault_triggering_tests)
print(str(len(bugs_with_failing_tests_in_commons_with_defects4j)) + " out of " + str(len(bugs_without_failing_tests_in_commons_with_defects4j) + len(bugs_with_failing_tests_in_commons_with_defects4j)) + " bugs have tests in common with defects4j (" + str(percentage_with_fault_triggering_tests) + "%)")

utils.dict_to_json_file(paths_dict["output_file_path"], output_dict)

Closure_9: 3
Closure_79: 2
Closure_32: 26
Closure_20: 2
Closure_86: 2
Closure_142: 570
Closure_115: 4
Closure_164: 5
Closure_131: 3
Closure_123: 4
Closure_65: 10
Closure_14: 2
Closure_41: 297
Closure_22: 2
Closure_30: 26
Closure_139: 562
Closure_127: 9
Closure_135: 368
Closure_80: 2
Closure_111: 4
Closure_26: 2
Closure_61: 2
Closure_24: 2
Closure_47: 25
Closure_28: 2
Closure_167: 3
Closure_175: 10
Closure_116: 4
Closure_50: 8
Closure_21: 2
Closure_74: 2
Closure_78: 2
Closure_23: 2
Closure_6: 192
Closure_153: 2
Closure_85: 2
Closure_89: 2
Closure_60: 2
Closure_56: 19
Closure_39: 4
Closure_173: 4
Closure_126: 9
Closure_157: 66
Closure_171: 5
Closure_70: 3
Closure_46: 2
Closure_29: 2
Compress_46: 61
Compress_10: 0
Compress_45: 61
Compress_41: 36
Compress_9: 1
JacksonCore_1: 2
JacksonCore_14: 4
JacksonDatabind_42: 34
JacksonDatabind_18: 27
Gson_11: 1
Codec_13: 0
Codec_7: 0
Codec_1: 1
Codec_3: 0
Cli_16: 20
Cli_32: 4
Cli_34: 0
Cli_8: 4
Cli_28: 4
Cli_24: 14
Cli_9: 4
Cli_39: 1
Cli_13: 32
Cli_2